In [ ]:
from pyincore import IncoreClient, MappingSet, RestorationService
from pyincore.analyses.electricpowerfacilityrestoration import ElectricPowerFacilityRestoration

In [ ]:
client = IncoreClient()

In [ ]:
epf_rest = ElectricPowerFacilityRestoration(client)
# Load restoration mapping
restorationsvc = RestorationService(client)
mapping_set = MappingSet(restorationsvc.get_mapping("61f302e6e3a03e465500b3eb"))  # new format of mapping
epf_rest.set_input_dataset('dfr3_mapping_set', mapping_set)
epf_rest.set_parameter("result_name", "epf_restoration.csv")
epf_rest.set_parameter("restoration_key", "Restoration ID Code")
epf_rest.set_parameter("end_time", 100.0)
epf_rest.set_parameter("time_interval", 1.0)
epf_rest.set_parameter("pf_interval", 0.01)

In [ ]:
epf_rest.run_analysis()

In [ ]:
pf_results = epf_rest.get_output_dataset("pf_results").get_dataframe_from_csv()
pf_results.head()

In [ ]:
time_results = epf_rest.get_output_dataset("time_results").get_dataframe_from_csv()
time_results.head()